# REvIEW
## Equivalent width calculator
***R***outine for
***Ev***aluating and 
***I***nspecting
***E***quivalent
***W***idths


Based on postage40mpfit.pro by Dr David Yong.

This notebook should output a .csv file and a plot for every equivelent width (EW) that you are fitting.



### Relavent files

There are two files which contain all the functions.

```REvIEW_plots.py```: concerns any functions in charge of creating the plots. These funcitons call the fitting functions within the plotting functions.

```REvIEW_fitting.py```: contains all the fitting functions. This includes the gradient preprocessing algorithm and the variable gaussian fitting functions. 

**NN directory**

note: needed to install
```bash
conda install pytorch torchvision torchaudio -c pytorch
``` 
to get the ML plug in to work

### Example files
To test REvIEW on your machine, you can use a subset of the spectra from two stars analysed in McKenzie et al. 2022 along with the corresponding line list. Compare the ```fit_outputs_example.csv``` to see if the EWs are in agreement. 

## Imports

Add any additional python imports that you might want here. 

There are two files which contain REvIEW functions. The only functions that are needed here are in the plotting module.

In [4]:
import os
import pandas as pd

# gen_plots generates the EW plots
# line_incrementer itterates through the line list,
# passing the lines that need to be plotted.
from REvIEW_plots import gen_plots, line_incrementer

### Directories

Define the location of the current REvIEW notebook (```review_path```), spectra (```spectra_path```), line list (```line_list_path```) and where you would like the plots of the EWs to be placed (```ew_path```). 

You will need to create a directory where you can put the ew .png and the output csv.


In [23]:
###  Edit these for your own system!!  ###

# The directory that contains this jupyter notebook
review_path    = os. getcwd()
# The directory that has your line list
line_list_path = os. getcwd() + '/examples/example_linelist'
# The directory that has your spectra (in .txt format)
spectra_path   = os. getcwd() + '/examples/example_spectra'
# The direcrtory where you would like the plots of EW to be placed
ew_path        = os. getcwd() + '/examples/example_ew'

### Spectra files

Pass a list of file names (```file_names```) that will be analysed. These must have the same names that appear in the ```spectra_path``` directory. The spectra must be continuum normalised. If they have also been shifted to rest frame wavelengths, enter 0 for ```rv_correction```.

Pass a list of the radial velovity corrections (```rv_correction```). The numbers must line up with the order that you listed your file names. 

It is reccomended that you run no more that **8** stars at a time, otherwise inspecting the plots afterwards becomes far more difficult. 

In [24]:
file_names = ['M22_III-3_example.txt', 'M22_III-14_example.txt']

rv_correction = [-122.7, -178.3]


# check that these spectra are in the directory you passed.
# If nothing is printed, REvIEW can find the spectra
os.chdir(spectra_path)
for file in file_names:
    if not os.path.isfile(file):
        print('warning, could not find file ', file)

## Line list
Add the name of the line list which appears in the ```line_list_path``` directory.

Import the line list as a pandas datafrane with the headdings 'wavelength' and 'species' (the code doesn't use any other columns). The current code assumes that there is no header lines in the file and the first two columns are the wavelength and species.

It's a good idea to look at the ```linelist``` pandas dataframe (create a new cell with only ```linelist``` to have the contense pop up). Some edits might need to be made if the line list has any header lines. 

In [25]:
linelist_name = "example_linelist.dat"

# Import the line list
os.chdir(line_list_path)
linelist = pd.read_table(linelist_name, sep="\s+", usecols = [0, 1],
                            names = ['wavelength', 'species'])

## Main celll for running REvIEW

### Variables
Three variables must be set:

```n_spectra:``` the number of spectra you are fitting to. Usually it's the same length as your ```file_names``` array but setting it to 1 is good for debugging/testing.

```lower_line_bound:``` the code will take any lines in the line list which are *higher* than this bound.

```upper_line_bound:``` the code will take any lines in the line list which are *lower* than this bound.

*note:* best to look at <=8 stars per run of review otherwise the plots become difficult to read. 

### Making the CSV
Creates a new csv with the name ```fit_outputs.csv``` if one doesn't already exist.
The csv is closed at the very end of the cell. If the code crashes within the main loop, nothing will be written to the csv.

### Main loop
The plotting function ```gen_plots``` from ```REvIEW_plots.py``` is called in the main loop which iterates through all the lines within the line bounds.

Once you have run this block once, check the file ```fit_outputs.csv``` to make sure things are looking normal. Any subsiquent evaluations of this cell will **append** to this csv, not overwrite it (i.e. if there is any bad data there that you don't want, delete this file and re-run this cell). 


In [26]:
# Set the number of stars to analyse
# (probably the number of filenames you entered)
n_spectra = len(file_names)



# What wavelength in the line list would you like to start?
lower_line_bound = 4780 # Angstroms
# What wavelength in the line list would you like to finish?
upper_line_bound = 4850 # Angstroms



# REvIEW will now analyse all lines between these wavelengths



# navigate to the directory where you will store your data
os.chdir(ew_path)

# Prepare the csv
if os.path.exists('fit_outputs.csv'):
    #just open
    output_csv  = open('fit_outputs.csv', "a+")
else:
    # open and write the headder
    output_csv  = open('fit_outputs.csv', "a+")
    # csv headder
    output_csv.write('line,star_name,flux_ratio,centre,sigma,ew,chi2,d,flag\n')


# Main loop which runs REvIEW
for line_info in line_incrementer(linelist, lower_line_bound, upper_line_bound):
    # Comment this line if you don't want to see which line you are up to in the line list. 
    print('line ', line_info[0])
    gen_plots(n_spectra, file_names, rv_correction, output_csv, spectra_path, ew_path, review_path, line_info[0], line_info[1], line_info[2], line_info[3])

# close the file
output_csv.close()

line  4786.807
line  4789.651
line  4794.36
line  4798.27
line  4800.649
line  4802.88
line  4808.16
line  4839.55
line  4848.88
